In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error 
import random
from sklearn.model_selection import train_test_split

import os
from tqdm import *

import sys
sys.path.insert(0, '../')

# tells which GPU to use.
os.environ["CUDA_VISIBLE_DEVICES"]="1"

np.random.seed(1017)
random.seed(1017)

import scipy.sparse as sp

import torch

from egexplainer import AttributionPriorExplainer

### We select only the genes that overlap with the MERGE score data

In [56]:
background_dataset = ExVivoDrugData(X_train, y_train) # another dataset class containing your training data
APExp = AttributionPriorExplainer(background_dataset, N,k=2)

# Build model

In [ ]:
import shap
from IPython.core.debugger import set_trace
from models import MLP

background = torch.FloatTensor(X_train.sample(100).values).cuda()
mse_term = torch.nn.MSELoss()

mean_train_losses = []
mean_valid_losses = []
epochs = 60

def train(prior_lambda):
    
    best_valid_loss = np.inf
    
    model = MLP().cuda().float()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

    print("Training model with prior penalty {}".format(prior_lambda))
    for epoch in range(epochs):
        model.train()

        train_losses = []
        valid_losses = []

        # Train
        for i, (features, labels) in enumerate(train_loader):
            features, labels = features.cuda().float(), labels.cuda().float()
            optimizer.zero_grad()
            outputs = model(features)

            eg = APExp.shap_values(model, features)
            eg_no_drugs = eg[:,len(drug_columns):]
            attribution_diff = eg_no_drugs - prior_features
            prior_penalty = prior_lambda*(attribution_diff ** 2).mean()

            #weight_penalty = 0.001*(torch.norm(model.layers[0].weight, p=1) + torch.norm(model.layers[2].weight, p=1) + torch.norm(model.layers[4].weight, p=1))
            loss = mse_term(outputs, labels) + prior_penalty

            loss.backward(retain_graph=True)
            optimizer.step()
            train_losses.append(loss.item())

        # Validation
        for i, (features,labels) in enumerate(test_loader):
            features, labels = features.cuda().float(), labels.cuda().float()
            outputs = model(features)
            valid_losses.append(mse_term(outputs,labels).detach().cpu().numpy())

            
        valid_loss = np.mean(valid_losses)
        
        if epoch % 10 == 0:
            print(epoch, np.mean(train_losses),valid_loss)
    return valid_loss


In [ ]:
prior_lambdas = np.linspace(1, 2, num = 20)
prior_lambdas

In [ ]:
prior_features = torch.FloatTensor(merge_df.loc[['Expression hub']].values).cuda().reshape(-1,)

In [ ]:
hubness_lambdas_and_loss = []

for prior_lambda in prior_lambdas:
    validation_loss = train(prior_lambda)
    hubness_lambdas_and_loss.append((prior_lambda, validation_loss))